In [1]:
import torch
from torch.utils.data import *
from transformers import *
import inspect
import sys
sys.path.insert(0, "..")

from models import *
from logic import *
from my_datasets import *

import numpy as np

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
n, r = 5, 8
ap, bp, tp = 0.2, 0.2, 0.4

train_qed_dataset_config = QedDatasetConfig(num_rules=r, num_vars=n, ante_prob=ap,
        conseq_prob=bp, theorem_prob=tp, num_items=640, base_seed=1234)
train_qed_dataset = QedDataset(train_qed_dataset_config)

test_qed_dataset = train_qed_dataset

# test_qed_dataset_config = QedDatasetConfig(num_rules=r, num_vars=n, ante_prob=ap,
#         conseq_prob=bp, theorem_prob=tp, num_items=100, base_seed=2345)
# test_qed_dataset = QedDataset(test_qed_dataset_config)

In [3]:
mytf_config = MyTfConfig(embed_dim=768, ffwd_width=1024, ffwd_depth=4, num_heads=2, num_layers=8)
mytf_model = get_seq2seq_model("mytf", config=mytf_config)
mytf_qed_config = QedTaskConfig(num_rules=r, num_vars=n, seq2seq_model=mytf_model)
mytf_qed_model = QedTaskModel(mytf_qed_config)

mygpt2_model = get_seq2seq_model("gpt2", use_pretrained=True)
mygpt2_qed_config = QedTaskConfig(num_rules=r, num_vars=n, seq2seq_model=mygpt2_model)
mygpt2_qed_model = QedTaskModel(mygpt2_qed_config)

loading configuration file config.json from cache at /home/antonxue/.cache/huggingface/hub/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.34.1",
  

In [4]:
training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    logging_steps=5
)

def compute_metrics(eval_preds):
    preds = eval_preds.predictions > 0
    avg_ones = np.mean(preds)
    acc = np.mean(preds == eval_preds.label_ids)
    return {"Accuracy" : acc, "Avg Ones" : avg_ones}

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [5]:
mygpt2_trainer = Trainer(
    model=mygpt2_qed_model,
    args=training_args,
    train_dataset = train_qed_dataset,
    eval_dataset = train_qed_dataset,
    # eval_dataset = test_qed_dataset,
    compute_metrics = compute_metrics)


In [ ]:
mygpt2_trainer.train()

***** Running training *****
  Num examples = 640
  Num Epochs = 10
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 270
  Number of trainable parameters = 126,421,249


logits tensor([-2.2176, -2.2768, -2.0203, -2.1876, -0.2425, -2.4516, -0.2454, -2.4084,
        -0.3088, -0.2123, -2.0911, -2.4125, -2.6518, -0.2024, -1.8057, -1.1964,
        -2.4911, -1.8280, -0.2723, -0.2985, -2.1845, -1.9740, -2.0620, -0.2969])
preds  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
labels tensor([0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]), acc 0.5416666865348816



Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.701400,0.720329,0.423438,0.000000
2,0.684700,0.693314,0.576562,1.000000


logits tensor([-0.1588, -1.2694, -1.2738, -1.3404, -1.2572, -0.6695, -1.5038, -0.1208,
        -1.3208, -1.5782, -1.3440, -0.1638, -0.1534, -0.5243, -0.1873, -1.2909,
        -1.6277, -0.1814, -1.0255, -1.0383, -0.1961, -1.3439, -0.9794, -1.4304])
preds  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
labels tensor([1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0]), acc 0.375

logits tensor([-0.6098, -0.4129, -0.7093, -0.7395, -0.4401, -0.1005, -0.1018, -0.0468,
        -0.0843, -0.7411, -0.7548, -0.1209, -0.0515, -0.3845, -0.8738, -0.3454,
        -0.0635, -0.5114, -0.7635, -0.6543, -0.2147, -0.0071, -0.2925, -0.0573])
preds  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
labels tensor([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]), acc 0.4583333432674408

logits tensor([ 0.0225, -0.0491,  0.0776, -0.0575, -0.1176, -0.0111

***** Running Evaluation *****
  Num examples = 640
  Batch size = 24


logits tensor([-0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518,
        -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518,
        -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518])
preds  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
labels tensor([0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1]), acc 0.5833333730697632

logits tensor([-0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518,
        -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518,
        -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518])
preds  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
labels tensor([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1]), acc 0.5

logits tensor([-0.2518, -0.2518, -0.2518, -0.2518, -0.2518, -0.2518, 

***** Running Evaluation *****
  Num examples = 640
  Batch size = 24


logits tensor([0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245,
        0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245,
        0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245])
preds  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
labels tensor([0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1]), acc 0.4166666865348816

logits tensor([0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245,
        0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245,
        0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245])
preds  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
labels tensor([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1]), acc 0.5

logits tensor([0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245, 0.6245,
        0.6245, 0.6245, 0.6245

In [7]:
batch = next(iter(DataLoader(train_qed_dataset, batch_size=8)))

In [8]:
mygpt2_qed_model.cpu().eval()
print(f"training? {mygpt2_qed_model.training}")
out = mygpt2_qed_model(**batch)

training? False
logits tensor([1.2405, 1.2405, 1.2405, 1.2405, 1.2405, 1.2405, 1.2405, 1.2405])
preds  tensor([1, 1, 1, 1, 1, 1, 1, 1])
labels tensor([0, 0, 1, 1, 0, 0, 1, 0]), acc 0.375



In [8]:
# mytf_trainer = Trainer(
#     mytf_qed_model,
#     training_args,
#     train_dataset = train_qed_dataset,
#     eval_dataset = test_qed_dataset,
#     compute_metrics = compute_metrics)

# mytf_trainer.train()

In [8]:
inspect.getfile(mygpt2_trainer.predict)

'/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/trainer.py'